In [ ]:
import json
import logging
import google.cloud.logging as cloud_logging
from googleapiclient import discovery
from google.oauth2 import service_account
from google.cloud import secretmanager

SECRET_ID     = 'VM_SERVICE_ACCOUNT'
PROJECT_ID    = "checkmate-453316"
ZONE          = "europe-west2-b"
INSTANCE_NAME = "chess-ingestion-vm"
MACHINE_TYPE  = "e2-micro"
VPC_NAME      = "filip-vpc"
SUBNET_NAME   = "filip-vpc"

def initialise_cloud_logger(PROJECT_ID):
    logging_client = cloud_logging.Client(project=PROJECT_ID)
    logging_client.setup_logging()
    logger = logging.getLogger(__name__)
    logger.propagate = True
    return logger

def gcp_access_secret(PROJECT_ID, SECRET_ID, VERSION_ID="latest"):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{PROJECT_ID}/secrets/{SECRET_ID}/versions/{VERSION_ID}"
    response = client.access_secret_version(name=name)
    return response.payload.data.decode('UTF-8')

def create_vm(request):

    # Initialise Logger Object
    logger = initialise_cloud_logger(PROJECT_ID)
    logger.info(f"Project: {PROJECT_ID} | Initialising VM Script and Deploying VM")

    service_account_creds = gcp_access_secret(PROJECT_ID, SECRET_ID)
    credentials = service_account.Credentials.from_service_account_file(service_account_creds)
    compute = discovery.build('compute', 'v1', credentials=credentials)

    config = {
        "name": INSTANCE_NAME,
        "machineType": f"zones/{ZONE}/machineTypes/{MACHINE_TYPE}",
        "disks": [{
            "boot": True,
            "initializeParams": {
                "sourceImage": "projects/debian-cloud/global/images/family/debian-12"
            }
        }],
        "networkInterfaces": [{
            "network": f"projects/{PROJECT_ID}/global/networks/{VPC_NAME}",
            "subnetwork": f"projects/{PROJECT_ID}/regions/{ZONE[:-2]}/subnetworks/{SUBNET_NAME}",
            "accessConfigs": [{
                "type": "ONE_TO_ONE_NAT",  # Required for external access
                "name": "External NAT"
            }]
        }]
    }

    operation = compute.instances().insert(
        project=PROJECT_ID,
        zone=ZONE,
        body=config
    ).execute()

    return logging.info(f"Instance {INSTANCE_NAME} creation started: {operation}")

In [21]:
request=""
create_vm(request)

"Instance chess-ingestion-vm creation started: {'kind': 'compute#operation', 'id': '2512460973966550698', 'name': 'operation-1742952516699-63134c19e4496-ac12bc05-55e510c0', 'zone': 'https://www.googleapis.com/compute/v1/projects/checkmate-453316/zones/europe-west2-b', 'operationType': 'insert', 'targetLink': 'https://www.googleapis.com/compute/v1/projects/checkmate-453316/zones/europe-west2-b/instances/chess-ingestion-vm', 'targetId': '4667222607258379946', 'status': 'RUNNING', 'user': 'startvm-sa@checkmate-453316.iam.gserviceaccount.com', 'progress': 0, 'insertTime': '2025-03-25T18:28:37.466-07:00', 'startTime': '2025-03-25T18:28:37.466-07:00', 'selfLink': 'https://www.googleapis.com/compute/v1/projects/checkmate-453316/zones/europe-west2-b/operations/operation-1742952516699-63134c19e4496-ac12bc05-55e510c0'}"